In [6]:
# imports
import numpy as np
import random
import pickle
import pandas as pd
import os

In [37]:
experiment_name = "test_10"
tokenizer = pickle.load(open(f"part_1/new_song2vec/experiments/{experiment_name}/data/tokenizer.pkl", "rb"))
embedding_weights = pickle.load(open(f"part_1/new_song2vec/experiments/{experiment_name}/embeddings.pkl", "rb"))
df = pd.read_csv('part_1/new_song2vec/raw_data/spotify_playlists.tsv', sep='\t', index_col=0)
random.sample(list(tokenizer.word_index.keys()), 5)

['spotify:track:3mgajkuu5qcekgu9aq2e2u',
 'spotify:track:3e7zwumjfqpsdojzekbrq7',
 'spotify:track:442j8vxab60dwf9cbfux5w',
 'spotify:track:08mfgeqevlf37eyz7jcwlc',
 'spotify:track:4at3d5qwnlibmvn75ecdrp']

In [29]:
def get_track_name(uri):
    return df.set_index("uri").loc[uri]['track_name']
def get_track_uri(name):
    return df.set_index('track_name').loc[name]['uri']

In [31]:
# function to get top-n most similar tracks
def get_most_similar_tracks(track_name, n=10, tokenizer=tokenizer, embedding_weights=embedding_weights):
    track_uri = get_track_uri(track_name)
    track_idx = tokenizer.word_index[track_uri]
    track_vector = embedding_weights[track_idx, :].reshape(1, -1)
    similarities = np.dot(track_vector, embedding_weights.T) / (np.linalg.norm(track_vector) * np.linalg.norm(embedding_weights, axis=1))
    similarities = similarities.reshape(-1)
    most_similar_idxs = np.argpartition(similarities, -(n+1))[-(n+1):]
    most_similar_idxs = most_similar_idxs[np.argsort(similarities[most_similar_idxs])][::-1][1:]

    # print most similar tracks, along with their positions in training data
    print("top {} tracks most similar to '{}' (pos. {}):".format(n, track_name, track_idx))
    for idx in most_similar_idxs:
        print("- (sim. {:.3f}): '{}' (pos. {})".format(similarities[idx], tokenizer.index_word[idx], idx))

In [36]:
uris = df["uri"].values
tokenizer.wor

10000

In [34]:
track_name = "STAY (with Justin Bieber)"
n = 100
song_list = get_most_similar_tracks(track_name, n=n)

KeyError: 'spotify:track:5HCyWlXZPP0y6Gqq8TgA20'

In [26]:
sorting_key = {song: index for index, song in enumerate(song_list)}

# Filter the DataFrame based on the song_list
filtered_df = df[df['track_name'].str.lower().isin(song_list)]

# Create a new column 'sorting_key' using the custom sorting key
filtered_df['sorting_key'] = filtered_df['track_name'].str.lower().map(sorting_key)

# Sort the DataFrame based on the custom sorting key
sorted_df = filtered_df.sort_values('sorting_key').drop('sorting_key', axis=1)

/var/folders/0s/3__sj1dn5hlcs4lg9h5cfgzh0000gn/T/ipykernel_10885/3120744123.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['sorting_key'] = filtered_df['track_name'].str.lower().map(sorting_key)


In [27]:
sorted_df

,artist_name,track_name,uri,num_plays,danceability,energy,key,loudness,mode,speechiness,...,instrumentalness,liveness,valence,tempo,type,id,track_href,analysis_url,duration_ms,time_signature
7619,Smino,blkswn,spotify:track:1BFFcF83SmsS6cKBLZGjkc,2,0.788,0.392,1.0,-8.944,0.0,0.2950,...,0.00000,0.0791,0.481,107.086,audio_features,1BFFcF83SmsS6cKBLZGjkc,https://api.spotify.com/v1/tracks/1BFFcF83SmsS...,https://api.spotify.com/v1/audio-analysis/1BFF...,172561.0,4.0
1945,David Guetta,Where Them Girls At (feat. Nicki Minaj & Flo R...,spotify:track:63rbP24OzfaQCJjsHWZXnY,32,0.664,0.872,3.0,-3.117,1.0,0.0396,...,0.00000,0.2280,0.555,129.873,audio_features,63rbP24OzfaQCJjsHWZXnY,https://api.spotify.com/v1/tracks/63rbP24OzfaQ...,https://api.spotify.com/v1/audio-analysis/63rb...,210093.0,4.0
4755,Eric Bellinger,9 Lives (Certified West Coast Killer) (feat. T...,spotify:track:773QYWdZHKUttjNOyffHlZ,5,0.647,0.719,1.0,-5.852,1.0,0.0851,...,0.00000,0.0944,0.390,93.955,audio_features,773QYWdZHKUttjNOyffHlZ,https://api.spotify.com/v1/tracks/773QYWdZHKUt...,https://api.spotify.com/v1/audio-analysis/773Q...,234880.0,4.0
7746,Henry Jackman,Big Hero 6,spotify:track:1WqFfYdK9ghVeMLTvpb6OJ,2,0.301,0.665,1.0,-7.966,1.0,0.0639,...,0.59600,0.0907,0.063,118.317,audio_features,1WqFfYdK9ghVeMLTvpb6OJ,https://api.spotify.com/v1/tracks/1WqFfYdK9ghV...,https://api.spotify.com/v1/audio-analysis/1WqF...,417893.0,4.0
4754,The Notorious B.I.G.,Hypnotize - 2014 Remaster,spotify:track:7KwZNVEaqikRSBSpyhXK2j,5,0.901,0.697,9.0,-3.439,1.0,0.2800,...,0.00000,0.2870,0.670,93.941,audio_features,7KwZNVEaqikRSBSpyhXK2j,https://api.spotify.com/v1/tracks/7KwZNVEaqikR...,https://api.spotify.com/v1/audio-analysis/7KwZ...,229827.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13359,Chloe x Halle,Down,spotify:track:6tATRoq8sbvKFHzvMncBc3,1,0.694,0.606,0.0,-6.060,1.0,0.0388,...,0.00000,0.1220,0.114,132.052,audio_features,6tATRoq8sbvKFHzvMncBc3,https://api.spotify.com/v1/tracks/6tATRoq8sbvK...,https://api.spotify.com/v1/audio-analysis/6tAT...,199120.0,4.0
1008,Jay Sean,Down,spotify:track:7LP4Es66zdY7CyjepqmvAg,58,0.657,0.695,2.0,-4.493,1.0,0.0321,...,0.00000,0.0822,0.683,65.997,audio_features,7LP4Es66zdY7CyjepqmvAg,https://api.spotify.com/v1/tracks/7LP4Es66zdY7...,https://api.spotify.com/v1/audio-analysis/7LP4...,212107.0,4.0
8715,Fifth Harmony,Down,spotify:track:7LEydI5NfjY6mZfKwYIptZ,1,0.530,0.512,8.0,-7.094,1.0,0.1330,...,0.00007,0.1280,0.401,196.121,audio_features,7LEydI5NfjY6mZfKwYIptZ,https://api.spotify.com/v1/tracks/7LEydI5NfjY6...,https://api.spotify.com/v1/audio-analysis/7LEy...,164985.0,4.0
1772,Mariah Carey,Fantasy (feat. O.D.B.),spotify:track:2itAOPLerxnnc8KXHMqPWu,35,0.837,0.548,2.0,-9.658,1.0,0.1160,...,0.00000,0.0500,0.821,102.497,audio_features,2itAOPLerxnnc8KXHMqPWu,https://api.spotify.com/v1/tracks/2itAOPLerxnn...,https://api.spotify.com/v1/audio-analysis/2itA...,290467.0,4.0
